In [84]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 64)
pd.set_option('display.max_rows', 100)
from sklearn.model_selection import GridSearchCV
import random

In [85]:
homePath = 'game'
trainPath = os.path.join(homePath, 'train_precessing.csv')
trainData = pd.read_csv(trainPath)
testPath = os.path.join(homePath, 'test_ALL.csv')
testData = pd.read_csv(testPath)
submPath = os.path.join(homePath, 'submission.csv')
submData = pd.read_csv(submPath)

In [86]:
trainData['emp_length'].fillna(trainData['emp_length'].median(), inplace=True)
trainData['annual_inc'].fillna(trainData['annual_inc'].median(), inplace=True)
trainData['title'].fillna(trainData['title'].median(), inplace=True)
trainData['pub_rec'].fillna(trainData['pub_rec'].median(), inplace=True)
trainData['revol_util'].fillna(trainData['revol_util'].median(), inplace=True)
trainData['total_acc'].fillna(trainData['total_acc'].median(), inplace=True)
trainData['collections_12_mths_ex_med'].fillna(trainData['collections_12_mths_ex_med'].median(), inplace=True)

In [87]:
testData['emp_length'].fillna(testData['emp_length'].median(), inplace=True)
testData['annual_inc'].fillna(testData['annual_inc'].median(), inplace=True)
testData['title'].fillna(testData['title'].median(), inplace=True)
testData['pub_rec'].fillna(testData['pub_rec'].median(), inplace=True)
testData['revol_util'].fillna(testData['revol_util'].median(), inplace=True)
testData['total_acc'].fillna(testData['total_acc'].median(), inplace=True)
testData['collections_12_mths_ex_med'].fillna(testData['collections_12_mths_ex_med'].median(), inplace=True)
del testData['member_id']

In [88]:
from sklearn.metrics import precision_score, recall_score
def calc_f2(label, predict):
    p = precision_score(label, predict)
    r = recall_score(label, predict)
    f2_score = 5*p*r / (4*p + r)
    return f2_score

In [89]:
trueData = trainData.loc[trainData['acc_now_delinq'] == 1]
trainData_copy = trainData.copy()
delLocal = np.array(np.where(np.array(trainData['acc_now_delinq']) == 1))
for i in delLocal.tolist():
    trainData_copy.drop(i, axis=0, inplace=True)
trueSize = len(trueData)

notBlankIndex = np.where(np.array(trainData_copy['member_id']) > 0)
subTrain = pd.concat([trueData, trainData_copy.loc[notBlankIndex][:3139]], axis=0, ignore_index=True)
randIndex = random.sample(range(0, len(subTrain)), len(subTrain))
subTrain = subTrain.loc[randIndex].reset_index()
notBlankIndex = np.where(np.array(subTrain['member_id']) > 0)
subTrain = subTrain.loc[notBlankIndex].reset_index()

E:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':


In [90]:
subLabel = subTrain['acc_now_delinq']
del subTrain['acc_now_delinq']
# subTrain = trainData
# subLabel = trainLabel

In [91]:
subLabel = list(map(int, subLabel))

In [92]:
# del subTrain['member_id']
# del trainData['member_id']
# # del subTrain['acc_now_delinq']
# trainData.head()
del subTrain['index']
del subTrain['member_id']

In [93]:
del subTrain['level_0']

In [94]:
# corr_matrx = subTrain.corr()
# corr_matrx['acc_now_delinq'].sort_values()

In [95]:
# from sklearn.ensemble import RandomForestClassifier
# # max_depth=13, min_samples_split=50, min_samples_leaf=20, max_features=9,
# rnd_clf = RandomForestClassifier(n_estimators=60,  min_samples_split=50, max_depth=13, max_features=9,  oob_score=True)
# rnd_clf.fit(subTrain, subLabel)
# rnd_pred = rnd_clf.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [96]:
# rnd_pred_prob = rnd_clf.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, rnd_pred))
# print(metrics.roc_auc_score(trainLabel, rnd_pred_prob))

In [97]:
# from sklearn.ensemble import BaggingClassifier
# bag_rnd = BaggingClassifier(rnd_clf, n_estimators=10, max_samples=1000, bootstrap=True, n_jobs=-1)
# bag_rnd.fit(subTrain, subLabel)
# rnd_pred = bag_rnd.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [98]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn import cross_validation, metrics
# gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
#                                  max_features='sqrt', subsample=0.7, random_state=10)
# gbdt.fit(subTrain, subLabel)
# y_pred = gbdt.predict(trainData)
# y_pred_prob = gbdt.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, y_pred))
# print(metrics.roc_auc_score(trainLabel, y_pred_prob))
# f2 = calc_f2(trainLabel, y_pred)
# f2

In [99]:
# from xgboost.sklearn import XGBClassifier
# xgb1 = XGBClassifier(learning_rate =0.05,
#                      n_estimators=1000,
#                      max_depth=3,
#                      min_child_weight=1,
#                      gamma=0.1,
#                      subsample=0.8,
#                      colsample_bytree=0.8,
#                      objective= 'binary:logistic',
#                      nthread=4,
#                      reg_alpha=0.001,
#                      reg_lambda=0.001,
#                      scale_pos_weight=1,
#                      seed=27)
# xgb1.fit(subTrain, subLabel)
# xgb1_pred = xgb1.predict(trainData)
# xgb1_pred_prob = xgb1.predict_proba(trainData)[:, 1]
# print(metrics.accuracy_score(trainLabel, xgb1_pred))
# print(metrics.roc_auc_score(trainLabel, xgb1_pred_prob))
# xgb1_f2 = calc_f2(trainLabel, xgb1_pred)
# print(xgb1_f2)

In [100]:
import lightgbm as lgb
# lgbTrainData = lgb.Dataset(subTrain, subLabel)
# param = {'lambda_l1':[0.5,0.8,1]}


In [101]:
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True,
                            random_state=10)


In [102]:
lgb_clf.fit(subTrain, subLabel)

LGBMClassifier(bagging_fraction=0.7, bagging_freq=6, boosting_type='gbdt',
        class_weight=None, colsample_bytree=0.7, feature_fraction=0.7,
        is_unbalance=True, lambda_l1=1, lambda_l2=0.001, learning_rate=0.1,
        max_depth=3, metric='auc', min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=450,
        min_gain_to_split=0.265, min_split_gain=0.0, n_estimators=1000,
        n_jobs=-1, num_leaves=5, objective='binary', random_state=10,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1, verbose=5)

In [103]:
# lgb.plot_importance(lgb_clf, figsize=(10,15))

In [104]:
# lgb_clf_pred = lgb_clf.predict(trainData)
# lgb_f2 = calc_f2(trainLabel, lgb_clf_pred)
# lgb_f2

In [105]:
# gsearch2 = GridSearchCV(estimator=lgb_clf, param_grid=param, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
# gsearch2.fit(subTrain, subLabel)
# print(gsearch2.best_params_)
# print(gsearch2.best_score_)

In [106]:
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier(learning_rate =0.05,
                     n_estimators=1000,
                     max_depth=3,
                     min_child_weight=1,
                     gamma=0.1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     reg_alpha=0.001,
                     reg_lambda=0.001,
                     scale_pos_weight=1)

import lightgbm as lgb
lgb_clf = lgb.LGBMClassifier(learning_rate=0.1,
                            boosting_type='gbdt',
                            objective='binary',
                            n_estimators=1000,
                            metric='auc',
                            max_depth=3,
                            num_leaves=5,
                            subsample=0.7,
                            colsample_bytree=0.7,
                            min_data_in_leaf=450,
                            feature_fraction=0.7,
                            bagging_fraction=0.7,
                            bagging_freq=6,
                            lambda_l1=1,
                            lambda_l2=0.001,
                            min_gain_to_split=0.265,
                            verbose=5,
                            is_unbalance=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.05, min_samples_split=320, min_samples_leaf=7, max_depth=7, 
                                 max_features='sqrt', subsample=0.7, random_state=10)

vot = VotingClassifier(estimators=[('xgb', xgb), ('lgb', lgb_clf)], voting='soft')

In [107]:
vot.fit(subTrain,subLabel)

VotingClassifier(estimators=[('xgb', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.1, learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='binary:logistic', reg_alpha=0.001, reg_lambda=0.0...da=0.0, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1, verbose=5))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [108]:
vot_pred = vot.predict(testData)


In [109]:
# from sklearn.ensemble import BaggingClassifier
# bag_rnd = BaggingClassifier(vot, n_estimators=10, max_samples=10000, bootstrap=True, n_jobs=-1)
# bag_rnd.fit(subTrain, subLabel)
# rnd_pred = bag_rnd.predict(trainData)
# rnd_f2 = calc_f2(trainLabel, rnd_pred)
# rnd_f2

In [110]:
submData['acc_now_delinq'] = vot_pred

In [111]:
submData.to_csv(submPath, index=False)

In [112]:
submData = pd.read_csv(submPath)
submData.head()

,member_id,acc_now_delinq
0,24344352,0
1,68831083,0
2,72767821,0
3,1539914,0
4,68359086,0


In [113]:
submData['acc_now_delinq'].value_counts()

0    149560
1     27916
Name: acc_now_delinq, dtype: int64

In [114]:
submData['acc_now_delinq']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        1
28        0
29        0
30        0
31        0
32        0
33        0
34        0
35        0
36        0
37        0
38        0
39        0
40        0
41        1
42        0
43        0
44        1
45        0
46        0
47        0
48        0
49        0
         ..
177426    0
177427    0
177428    0
177429    0
177430    0
177431    0
177432    0
177433    0
177434    1
177435    1
177436    0
177437    0
177438    0
177439    0
177440    0
177441    0
177442    0
177443    1
177444    0
177445    0
177446    1
177447    1
177448    1
177449    0
177450    1
177451    1
177452    0
177453    0
177454    0
177455    1
177456    0
177457    0
1774